<h1> Capstone 2 Data Wrangling </h1>

The data is from: https://www.kaggle.com/c/kkbox-churn-prediction-challenge/overview/evaluation 

In [2]:
# import necessary libraries and packages.
# we will be using dask to read data as we are dealing with large files 
import dask.dataframe as dd
import numpy as np

# Read in train_v2.csv

This dataset has our target value, is_churn, and a unique identifier for each customer, msno.

In [4]:
# read csv file with target data: train_df
train_df = dd.read_csv("../data/raw/train_v2.csv")

In [5]:
train_df.head()

,msno,is_churn
0,ugx0CjOMzazClkFzU2xasmDZaoIqOUAZPsH1q0teWCg=,1
1,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1
4,K6fja4+jmoZ5xG6BypqX80Uw/XKpMgrEMdG2edFOxnA=,1


In [6]:
# Let's explore our target column, is_churn:
print("Number of missing values: ", sum(train_df.is_churn.isna()))
train_df.is_churn.value_counts().compute()

Number of missing values:  0


0    883630
1     87330
Name: is_churn, dtype: int64

In [7]:
# Number of rows and unique ids in the dataset: 
print("Number of rows:", f'{len(train_df):,}')
print("Number of unique msno ids:", f'{len(train_df.msno.unique()):,}')

Number of rows: 970,960
Number of unique msno ids: 970,960


# Read in members_v3.csv

This dataset has a unique identifier for each member, msno, as well as some demographic information. 

In [8]:
# read in the members dataframe: members_df
members_df = dd.read_csv("../data/raw/members_v3.csv")

In [9]:
members_df.head()

,msno,city,bd,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,20110915
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,20110915
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915


In [10]:
members_df.describe()

,city,bd,registered_via,registration_init_time
npartitions=1,,,,
,float64,float64,float64,float64
,...,...,...,...


In [39]:
# the 'bd' column is a little unclear. From reading the information provided by the data source,
# I see that it is actually just the age of the customer. I will change this for clarity.
members_df = members_df.rename(columns={'bd':'age'})
members_df.head()

,msno,city,age,gender,registered_via,registration_init_time
0,Rb9UwLQTrxzBVwCB6+bCcSQWZ9JiNLC9dXtM1oEsZA8=,1,0,NaN,11,20110911
1,+tJonkh+O1CA796Fm5X60UMOtB6POHAwPjbTRVl/EuU=,1,0,NaN,7,20110914
2,cV358ssn7a0f7jZOwGNWS07wCKVqxyiImJUX6xcIwKw=,1,0,NaN,11,20110915
3,9bzDeJP6sQodK73K5CBlJ6fgIQzPeLnRl0p5B77XP+g=,1,0,NaN,11,20110915
4,WFLY3s7z4EZsieHCt63XrsdtfTEmJ+2PnnKLH5GY4Tk=,6,32,female,9,20110915


In [12]:
# Number of rows and unique ids in the dataset: 
print("Number of rows:", f'{len(members_df):,}')
print("Number of unique msno ids:", f'{len(members_df.msno.unique()):,}')

Number of rows: 6,769,473
Number of unique msno ids: 6,769,473


# Read in transactions_v2.csv

This dataset is a record of each customer's transactions. 

In [13]:
# read in the transactions csv file: transactions_df
transactions_df = dd.read_csv("../data/raw/transactions_v2.csv")

In [14]:
transactions_df.head()

,msno,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,++6eU4LsQ3UQ20ILS7d99XK8WbiVgbyYL4FUgzZR134=,32,90,298,298,0,20170131,20170504,0
1,++lvGPJOinuin/8esghpnqdljm6NXS8m8Zwchc7gOeA=,41,30,149,149,1,20150809,20190412,0
2,+/GXNtXWQVfKrEDqYAzcSw2xSPYMKWNj22m+5XkVQZc=,36,30,180,180,1,20170303,20170422,0
3,+/w1UrZwyka4C9oNH3+Q8fUf3fD8R3EwWrx57ODIsqk=,36,30,180,180,1,20170329,20170331,1
4,+00PGzKTYqtnb65mPKPyeHXcZEwqiEzktpQksaaSC3c=,41,30,99,99,1,20170323,20170423,0


In [15]:
# check the data types of each column:
transactions_df.describe()

,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
npartitions=1,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...


In [16]:
# Number of rows and unique ids in the dataset: 
print("Number of rows:", f'{len(transactions_df):,}')
print("Number of unique msno ids:", f'{len(transactions_df.msno.unique()):,}')

Number of rows: 1,431,009
Number of unique msno ids: 1,197,050


# Read in user_logs_v2.csv

This dataset is a log of a user's activity.

msno: user id

date: format %Y%m%d

num_25: number of songs played less than 25% of the song length

num_50: number of songs played between 25% to 50% of the song length

num_75: number of songs played between 50% to 75% of of the song length

num_985: number of songs played between 75% to 98.5% of the song length

num_100: number of songs played over 98.5% of the song length

num_unq: number of unique songs played

total_secs: total seconds played

In [17]:
# read in the user_logs csv file: user_logs_df
user_logs_df = dd.read_csv("../data/raw/user_logs_v2.csv")

In [18]:
user_logs_df.head()

,msno,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,u9E91QDTvHLq6NXjEaWv8u4QIqhrHk72kE+w31Gnhdg=,20170331,8,4,0,1,21,18,6309.273
1,nTeWW/eOZA/UHKdD5L7DEqKKFTjaAj3ALLPoAWsU8n0=,20170330,2,2,1,0,9,11,2390.699
2,2UqkWXwZbIjs03dHLU9KHJNNEvEkZVzm69f3jCS+uLI=,20170331,52,3,5,3,84,110,23203.337
3,ycwLc+m2O0a85jSLALtr941AaZt9ai8Qwlg9n0Nql5U=,20170331,176,4,2,2,19,191,7100.454
4,EGcbTofOSOkMmQyN1NMLxHEXJ1yV3t/JdhGwQ9wXjnI=,20170331,2,1,0,1,112,93,28401.558


In [19]:
# Use describe to check data types
user_logs_df.describe()

,date,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
npartitions=1,,,,,,,,
,float64,float64,float64,float64,float64,float64,float64,float64
,...,...,...,...,...,...,...,...


In [20]:
# Number of rows and unique ids in the dataset: 
print("Number of rows:", f'{len(user_logs_df):,}')
print("Number of unique msno ids:", f'{len(user_logs_df.msno.unique()):,}')

Number of rows: 18,396,362
Number of unique msno ids: 1,103,894


# Joining tables

<h2>First, we will join train_df and transactions_df: train_trans_df </h2>

In [21]:
# Each dataframe has an 'msno' column which we will use to join the dataframes. 
# Let's start by finding the intersection of train_df and transactions_df, train_trans_df:
join_train_trans = dd.merge(train_df, transactions_df, on='msno', how='inner')

In [22]:
join_train_trans.head()

,msno,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,transaction_date,membership_expire_date,is_cancel
0,f/NmvEzHfhINFEYZTR05prUdr+E+3+oewvweYz9cCQE=,1,36,30,180,180,0,20170311,20170411,0
1,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,17,60,0,0,0,20170311,20170314,0
2,zLo9f73nGGT1p21ltZC3ChiRnAVvgibMyazbCxvWPcg=,1,15,90,300,300,0,20170314,20170615,0
3,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41,30,149,149,1,20150908,20170608,0
4,8iF/+8HY8lJKFrTc7iR9ZYGCG2Ecrogbc2Vy5YhsfhQ=,1,41,30,149,149,1,20150908,20170708,0


In [23]:
# let's see how many rows we have:
f'{len(join_train_trans):,}'

'1,132,036'

This is more rows than the 970,060 unique msno's we have from train_df. This means that we have some customers who have multiple transactions. The most recent transaction should be most valuable when it comes to predicting churn, so I will isolate the most recent transaction.

In [24]:
# Group train_trans_df by msno and choose the most recent transaction_date.
# Since the transaction_date is stored as a float64 as YYYYMMDD, I can just choose
# the maximum value of transaction_date to find the most recent transaction
most_recent_trans = join_train_trans.groupby("msno")["transaction_date"].max()
f'{len(most_recent_trans):,}'

'933,578'

This is less than the total 970,060 customers we have in our target dataframe, train_df, which means that we don't have the transaction data for some of those customers. However, we are only in customers that we do have a "is_churn" value for, so we will ignore those other customers.

In [25]:
# Filter the train_trans_df so that we only keep the rows that have:
#  - a value for the "is_churn" column
#  - the most recent transaction for each customer

most_recent_trans = most_recent_trans.to_frame().reset_index()
train_trans_df = dd.merge(most_recent_trans, join_train_trans, on=["msno", "transaction_date"], how="inner")
train_trans_df.head()

,msno,transaction_date,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,membership_expire_date,is_cancel
0,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,20170331,0,39,30,149,149,1,20170519,0
1,++/9R3sX37CjxbY/AaGvbwr3QkwElKBCtSvVzhCBDOk=,20170315,0,41,30,149,149,1,20170415,0
2,++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=,20170331,0,14,30,149,149,1,20170504,0
3,++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=,20170320,0,41,30,149,149,1,20170420,0
4,++0nOC7BmrUTtcSboRORfg6ZXTajnBDt1f/SEgH6ONo=,20170306,0,40,30,149,149,1,20170414,0


In [26]:
f'{len(train_trans_df):,}'

'945,268'

In [27]:
f'{len(train_trans_df.msno.unique()):,}'

'933,578'

There must be some customers with multiple transactions on the same day... let's check if this is true. 

In [28]:
# create a column of just 1s
train_trans_df["ones"] = 1

In [29]:
# Group by msno and sum the 'ones' column. 
find_duplicates = train_trans_df.groupby("msno").ones.sum()
find_duplicates = find_duplicates.to_frame()

# Find any rows with values greater than 1 in the 'ones' column.
find_duplicates[find_duplicates.ones > 1].compute()

,ones
msno,
++kosgi4V03jOxcBKjM/9tPignUOxcc7jBVnZLJ+lX0=,2
++r9mKptLWw8f5X2H/AsUYwzQgYtAVJQNEp5qrLEhQ8=,2
+0hXEuyLuw+Kgfa4BUz8v3R27ZmWPWKSktfe9twCVhA=,2
+0vlUFgYzPhr9uYW4pOrJ0XJ2YajWybzRQm4gQ7eRSk=,3
+1T61obWBryWTi8EKj9iG7LiwGxe7iAkItWMy2w+G5k=,2
...,...
zjL52LqD3ASp/TpZHk1skvmZxT7I9qm18stOLTZR5oQ=,3
zkg67+dNQezt9xpPCzIHV8XTtF5DEafCe2/dgjgobHI=,2
zmr2mZ7IWsnmsbYVBTWZhxpLuSvCDvJ1Sal0jkQJN7g=,2


So there are 9,283 customers with multiple transactions in one day.


__Question: Will this effect my conclusions? I see two possibilites:__
    
    1) The question is whether or not an *individual customer* will renew, so each customer should have a unique row with a churn value associated with it. 
    
    2) The question is whether a *particular set of features* will lead to churn, so it does not matter if there is a unique customer associated with each set of features. 
    
I think option 2 makes more sense, because for making a prediction, the features are more important than the customer's individual identity. The msno column will likely be dropped before training the model anyways. 


In [31]:
train_trans_df = train_trans_df.drop(columns=["ones"])

<h2> Join train_trans_df and members_df: join_mem_df </h2>

Above, we saw that every msno in members_df is unique. So this join is more straightforward than the previous one. We are simply adding the biographical data in members_df for each of the entries. 

In [32]:
join_mem_df = dd.merge(train_trans_df, members_df, on='msno', how='inner')
join_mem_df.head()

,msno,transaction_date,is_churn,payment_method_id,payment_plan_days,plan_list_price,actual_amount_paid,is_auto_renew,membership_expire_date,is_cancel,city,age,gender,registered_via,registration_init_time
0,++/ZHqwUNa7U21Qz+zqteiXlZapxey86l6eEorrak/g=,20170331,0,14,30,149,149,1,20170504,0,13,39,male,3,20140421
1,++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=,20170320,0,41,30,149,149,1,20170420,0,1,0,NaN,7,20160320
2,++0nOC7BmrUTtcSboRORfg6ZXTajnBDt1f/SEgH6ONo=,20170306,0,40,30,149,149,1,20170414,0,13,25,male,9,20100203
3,++A8p4GrsTnMjI6hAZEtlRsaz6s6O9ddUoH0fmS4s7s=,20170326,0,30,30,149,149,1,20170426,0,5,43,female,9,20141118
4,++F4A2D3J5tCLrRY8t2HU8VpfLu8R8t4czTbbRm5/78=,20170309,0,41,30,99,99,1,20170409,0,1,0,NaN,7,20151109


In [33]:
f'{len(join_mem_df):,}'

'836,632'

#We see that the length of this dataset is about 100,000 less than train_trans_df. This means that for whatever reason, there are some customers who are not listed in members_df that are in train_trans_df. We are only interested in the intersection of the datasets, as we are looking to get as many customers as we can with all the data. 

<h2> Finally, merge join_mem_df and user_logs_df: full_df </h2>

Before we merge the datasets, we want to sum up the data in user_logs so that we can get a picture of a customer's habits for the full duration of their membership, not just in a particular day.

In [41]:
# First, we will convert the date column to 1s so we can get the total number of days
all_time_df = user_logs_df.copy()
all_time_df.date = 1
all_time_df = all_time_df.rename(columns={"date":"total_days"})

In [42]:
# Next, we will group by msno and aggregate using sum
all_time_df = all_time_df.groupby('msno').sum()
all_time_df = all_time_df.reset_index()
all_time_df.head()

,msno,total_days,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs
0,+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=,26,86,11,10,5,472,530,117907.425
1,+++hVY1rZox/33YtvDgmKA2Frg/2qhkz12B9ylCvh8o=,31,191,90,75,144,589,885,192527.892
2,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,28,43,12,15,12,485,468,115411.260
3,++0+IdHga8fCSioOVpU8K7y4Asw8AveIApVH2r9q9yY=,25,190,34,21,20,331,582,90177.554
4,++0/NopttBsaAn6qHZA2AWWrDg7Me7UOMs1vsyo4tSI=,8,21,8,17,7,104,115,28450.268


In [43]:
f'{len(all_time_df):,}'

'1,103,894'

In [47]:
# Finally, we can merge join_mem_df and all_time_df to get full_df:
full_df = dd.merge(join_mem_df, all_time_df, on='msno', how='inner')

In [50]:
# Let's get the shape of our final dataframe:
print("rows:", f'{len(full_df):,}')
print("columns:", len(full_df.columns))

rows: 736,230
columns: 23


In [52]:
full_df.head(1).T

,0
msno,++hTNyKbQJonbwH4zStU+NGBhqxsUwQ++qQwLaZJliA=
transaction_date,20170331
is_churn,0
payment_method_id,39
payment_plan_days,30
plan_list_price,149
actual_amount_paid,149
is_auto_renew,1
membership_expire_date,20170504
is_cancel,0
